In [1]:
import collections
import math, random
import itertools
import os, copy
from copy import deepcopy
import time
import pandas as pd

def initializeBoard():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.randint(0,1)
            z = -x-y
            Hex[(x, y, z)] = ''#side[rnd]
    return Hex

def allWhiteBoardZobrist():
    whitezobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            whitezobristboard[(x, y, z)] = rnd
    return whitezobristboard

def allBlackBoardZobrist():
    blackzobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            blackzobristboard[(x, y, z)] = rnd
    return blackzobristboard

def initializeBoardZobrist():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            Hex[(x, y, z)] = rnd
    return Hex

def boardToZobrist(board, zboard, wzobristboard, bzobristboard):
    random.seed(1356)
    zobristkey = random.getrandbits(64)
    wzobrists = wzobristboard.values()
    bzobrists = bzobristboard.values()
    
    for cel in board:
        if board[cel] == 'b':
            zobristkey ^= (bzobristboard[cel] ^ zboard[cel])
        elif board[cel] == 'w':
            zobristkey ^= (wzobristboard[cel] ^ zboard[cel])
        
    return zobristkey

In [2]:
# Form groups and calculate nr of members
def EvalBoard(board):
    score = find_scores(board, 'b') #score of player1
    score2 = find_scores(board, 'w') #score of player2

    group = findGroups(score)
    group2 = findGroups(score2)

    player_one_total, player_two_total = result(group, group2)
    tempscore = 0
  
    return player_one_total, player_two_total, player_one_total - player_two_total

def find_scores(board, side):   
    tot = {}
    
    listOfkeys = [key for (key, value) in board.items() if value == side]
    
    for lok  in listOfkeys:
        tot[lok] = side
        
    return tot

def listOfEmptySpots(board):
    return [k for k, v in board.items() if v == '']

def neighbours(tiles, scoregroup, visited):
    group = []
    for tile in tiles:
        for direction in hex_directions:
            new_tile2 = tuple(map(sum, zip(tile, direction)))
            if new_tile2 in scoregroup and new_tile2 not in visited:
                group += [new_tile2]
                visited += [new_tile2]
    return group, visited

def findGroups(score):
    newlist = []
    visited = []
    for sc in score:
        temp = [sc]
        if sc not in visited:
            visited += [sc]
            groupt, visited = neighbours([sc], score, visited)
            temp += groupt
            visited += groupt
            find = True
            while find:
                groupt, visited = neighbours(groupt, score, visited)
                if len(groupt) != 0:
                    temp += groupt
                else:
                    find = False
                    
            newlist += [len(temp)]
    return newlist


# multiply all group counts of the same color for final game score
from functools import reduce

def multiplyall(count_list):
    if count_list != []:
        return reduce(lambda x, y: x*y, count_list)
    return 0

def result(count_list, count_list2):
    game_total = multiplyall(count_list)
    game_total2 = multiplyall(count_list2)
    return game_total, game_total2

# Put pieces on board
def makeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
    return board

# Put pieces on board
def unmakeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
    return board

def isEmpty(board, position):
    if position in board:
        return len(board[position]) == 0
    else:
        return False

def fullBoard(board):
    number_ofcells = len(board)
    number_ofstones = 0
    for cell in board:
        if board[cell] != '':
            number_ofstones += 1
    return number_ofstones >= number_ofcells -1

#Check if a move is valid
def validMove(board, x, y):
    
    #Check if there is a piece on the location
    if not isEmpty(board, (x, y, -x-y)):
        return False
    
    #Check if the piece is on the board
    if abs(x) > N or abs(y) > N:
        return False
    
    return True

In [3]:
def moves_left(board):
    print('These are the moves left: ')
    
    moves = ''
    counter = 0
    for pos in board:
        
        if board[pos] == '':
            moves += str(pos)
            counter += 1
            if counter == 7:
                moves += '\n'
                counter = 0
            else:
                moves += ' '
    print (moves)

In [4]:
def printBoard(board):
    for x in range(-N, N+1):
        string = ''
        if x > 0:
            string += ' ' * (N-(N-x))
        else:
            string += ' ' * -(N-(N-x))
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):

            if (x, y, -x-y) in board:
                if board[(y, -x-y, x)] == '':
                    string += '_ '
                else:
                    string += board[(y, -x-y, x)] + ' '
        print(string)      

In [5]:
# if no valid moves possible then True
def leafNode(board):
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    
    bestValue = -INFINITY
    for cell in all_cells:
        x, y, _ = cell[0]
        x2, y2, _ = cell[1]
        #Make two valid moves
        if validMove(board, x, y) and validMove(board, x2, y2):
            return False
    return True

In [6]:
def zobristStateScore(board):

    # Create a zobrist key for the new board/childnode
    zidentity = boardToZobrist(board, zboard, wzobristboard, bzobristboard)
    maxplayer, minplayer, tempscore = 0, 0, 0

    maxplayer, minplayer, tempscore = EvalBoard(board)
        
    return zidentity%1000, tempscore, maxplayer, minplayer

In [7]:
# random.seed(1356) # set seed so that each play is the same

hex_directions = [(1, 0, -1), (1, -1, 0), (0, -1, 1), (-1, 0, 1), (-1, 1, 0), (0, 1, -1)]
hex_diagonals = [(2, -1, -1), (1, -2, 1), (-1, -1, 2), (-2, 1, 1), (-1, 2, -1), (1, 1, -2)]

side = [random.getrandbits(64), random.getrandbits(64)]#[b, w, empty] # side to move ['b', 'w']#

N = 4 # Number of hexes side
p = 2 # number of players

blackstones = 136
whitestones = 136

#Game ends if number of free tiles < 4
total_nr_stones = blackstones + whitestones <4

board = initializeBoard() # board with assigned black and white pieces
zboard = initializeBoardZobrist() # initial board, without any pieces

wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

INFINITY = 999

# 1) Minimax Alpha Beta pruning

In [8]:
def AlphaBeta(board, depth, alpha, beta, MAX):
    if depth == 0 and fullBoard(board):
        if MAX:
            return EvalBoard(board)[0]
        else:
            return EvalBoard(board)[1]
    
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_temp = deepcopy(board) # make sure that you copy the board from the parent!!
    if MAX:
        bestValue = -INFINITY
        for cell in all_cells:
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
                makeMove(board_temp, [x, y], 'w')
                makeMove(board_temp, [x2, y2], 'b')
                bestValue = max(bestValue, AlphaBeta(board_temp, depth-1, alpha, beta, False))
                
                alpha = max(alpha, bestValue)
                if beta <= alpha:
                    all_cells = all_cells[:all_cells.index(cell)]
                    break # beta cut-off
        return bestValue
    else: # MIN
        bestValue = INFINITY
        for cell in all_cells:
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
                makeMove(board_temp, [x, y], 'w')
                makeMove(board_temp, [x2, y2], 'b')
                bestValue = min(bestValue, AlphaBeta(board_temp, depth-1, alpha, beta, True))
                
                beta = min(beta, bestValue)
                if beta <= alpha:
                    all_cells = all_cells[:all_cells.index(cell)]
                    break # alpha cut-off
        return bestValue

# 2) Minimax Alpha Beta pruning with transposition table

In [9]:
def AlphaBetaTT(board, depth, ply, alpha, beta, MAX):
    if leafNode(board) or len(listOfEmptySpots(board)) < 2 or depth == 0:
        if MAX:
            return EvalBoard(board)[0]
        else:
            return EvalBoard(board)[1]
        
    global node_count;
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_temp = deepcopy(board) # make sure that you copy the board from the parent!!

    if MAX:
        v = -INFINITY
        for cell in all_cells:
            node_count += 1
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
                makeMove(board_temp, [x, y], 'w')
                makeMove(board_temp, [x2, y2], 'b')
                
                boardZobrist = boardToZobrist(board_temp, zboard, wzobristboard, bzobristboard)
                index = boardZobrist % 1000 #% len(tt)

                if index in tt:
                    continue
                
                index, score, maxplayer, _ = zobristStateScore(board_temp)

                tt[index] = [depth, MAX, maxplayer]
                value = AlphaBetaTT(board_temp, depth-1, ply+1, alpha, beta, False)
                v = max(v, value)
                alpha = max(alpha, v)
                
                if beta <= alpha:
                    all_cells = all_cells[:all_cells.index(cell)]
                    break # beta cut-off
        return v
    else: # MIN
        v = INFINITY
        for cell in all_cells:
            node_count += 1
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
                makeMove(board_temp, [x, y], 'w')
                makeMove(board_temp, [x2, y2], 'b')

                # Search for the key in the transposition table
                # 
                boardZobrist = boardToZobrist(board_temp, zboard, wzobristboard, bzobristboard)
                index = boardZobrist % 1000 #% len(tt)

                if index in tt:
                    continue

                index, score, _, minplayer = zobristStateScore(board_temp)

                tt[index] = [depth, MAX, minplayer]
            
                value = AlphaBetaTT(board_temp, depth-1, ply+1, alpha, beta, True)
                v = min(v, value)
                
                beta = min(beta, v)
                if beta <= alpha:
                    all_cells = all_cells[:all_cells.index(cell)]
                    break # alpha cut-off
        return v

# 3) Negamax Alpha Beta pruning

In [10]:
import itertools
from copy import deepcopy

def NegamaxAB(board, depth, alpha, beta, side):
    if leafNode(board) or len(listOfEmptySpots(board)) < 2 or depth == 0:
        return -side * EvalBoard(board)[2]
    global node_count;
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_w = deepcopy(board) # make sure that you copy the board from the parent!!
    
    bestValue = -INFINITY
    for cell in all_cells:
        node_count += 1
        x, y, _ = cell[0]
        x2, y2, _ = cell[1]

        #Make two valid moves
        if validMove(board_w, x, y) and validMove(board_w, x2, y2):
            makeMove(board_w, [x, y], 'w')
            makeMove(board_w, [x2, y2], 'b')
        
            v = -NegamaxAB(board_w, depth-1, -beta, -alpha, -side)

            bestValue = max(bestValue, v)
            alpha = max(alpha, v)
            
            if alpha >= beta:
                all_cells = all_cells[:all_cells.index(cell)]
                break
    
    return bestValue

# 4) Negamax Alpha Beta pruning with transposition table lookup

In [11]:
import itertools
from copy import deepcopy

def NegamaxABTT(board, depth, ply, alpha, beta, side):
    if leafNode(board) or len(listOfEmptySpots(board)) < 2 or depth == 0:
        return -side * EvalBoard(board)[2]
    global node_count;
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_w = deepcopy(board) # make sure that you copy the board from the parent!!
    
    bestValue = -INFINITY
    for cell in all_cells:
        x, y, _ = cell[0]
        x2, y2, _ = cell[1]
        node_count += 1
        #Make two valid moves
        if validMove(board_w, x, y) and \
            validMove(board_w, x2, y2):
                makeMove(board_w, [x, y], 'w')
                makeMove(board_w, [x2, y2], 'b')

                # Search for the key in the transposition table
                # 
                boardZobrist = boardToZobrist(board_w, zboard, wzobristboard, bzobristboard)
                index = boardZobrist % 1000 #% len(tt)

                if index in tt:
                    continue

                index, score, _, _ = zobristStateScore(board_w)

                tt[index] = [depth, -side, score]
            
                value = -NegamaxABTT(board_w, depth-1, ply+1, alpha, beta, -side)
                bestValue = min(bestValue, value)
                
                alpha = min(alpha, value)
                if alpha >= beta:
                    all_cells = all_cells[:all_cells.index(cell)]
    return bestValue

# 5) Negascout (PVS) minimal window

In [12]:
def Negascout(board, depth, alpha, beta, side):
    if leafNode(board) or len(listOfEmptySpots(board)) < 2 or depth == 0:
        return -side * EvalBoard(board)[2]
    
    first_child = True
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_temp = deepcopy(board) # make sure that you copy the board from the parent!!
    
    for cell in all_cells:
        x, y, _ = cell[0]
        x2, y2, _ = cell[1]
        
        #Make two valid moves
        if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
            makeMove(board_temp, [x, y], 'w')
            makeMove(board_temp, [x2, y2], 'b')
            
            if not first_child:
                score = -Negascout(board_temp, depth-1, -alpha-1, -alpha, -side)
                if alpha < score and score < beta:
                    score = -Negascout(board_temp, depth-1, -beta, -score, -side)
            else:
                first_child = False
                score = -Negascout(board_temp, depth-1, -beta, -alpha, -side)
                
            alpha = max(alpha, score)
            if alpha >= beta:
                all_cells = all_cells[:all_cells.index(cell)]
                break
    return alpha

# For each selected algorithm, find the best moves per state

In [13]:
def BestMove(board):
    maxPoints = 0
    mx = N+1; 
    my = N+1
    mx2 =  N+1; 
    my2 = N+1
    
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_temp = deepcopy(board) # make sure that you copy the board from the parent!!
    
    while len(all_cells) > 0:
        rnd = random.randint(0, len(all_cells)-1)
        cell = all_cells[rnd]
        x, y, _ = cell[0]
        x2, y2, _ = cell[1]

        #Make two valid moves
        if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
            makeMove(board_temp, [x, y], 'w')
            makeMove(board_temp, [x2, y2], 'b')

            if opt == 0:
                points = AlphaBeta(board_temp, depth, -INFINITY, +INFINITY, True)
            elif opt == 1:
                  
                # Search for the key in the transposition table
                # 
                boardZobrist = boardToZobrist(board_temp, zboard, wzobristboard, bzobristboard)
                index = boardZobrist % 1000
                if index in tt:
                    points = tt[index][-1]
                
                    if points >= maxPoints:
                        maxPoints = points
                        mx = x
                        my = y
                        mx2 = x2
                        my2 = y2
                        continue
                        
                points = AlphaBetaTT(board_temp, depth, ply, -INFINITY, +INFINITY, True)
                
            elif opt == 2:
                points = NegamaxAB(board_temp, depth, -INFINITY, +INFINITY, 1)
            elif opt == 3:               
                
                # Search for the key in the transposition table
                # 
                boardZobrist = boardToZobrist(board_temp, zboard, wzobristboard, bzobristboard)
                index = boardZobrist % 1000
                if index in tt:
                    points = tt[index][-1]
                
                    if points >= maxPoints:
                        maxPoints = points
                        mx = x
                        my = y
                        mx2 = x2
                        my2 = y2
                        continue
                        
                points = NegamaxABTT(board_temp, depth, ply, -INFINITY, +INFINITY, 1)

#             Replace best move
            if points >= maxPoints:
                maxPoints = points
                mx = x
                my = y
                mx2 = x2
                my2 = y2
        all_cells.remove(cell)
    
    return [mx, my], [mx2, my2]

# Play the game

In [15]:
# def startGame(play):
tt = {}
gamehistory = {}
points = 0
ply = 0
node_count = 0
def menuOptions(xy):
    if xy == '':
        os._exit(0)
        
    if xy == '+': 
        unmakeMove(board, [int(x), int(y)], 'b')
        unmakeMove(board, [int(x2), int(y2)], 'w')
        print('Undone', [int(x), int(y)], [int(x2), int(y2)])
        xy = input('X,Y: ')

    # Check available moves
    if xy == '?': 
        moves_left(board)
        xy = input('X,Y: ')
        
print ('Omega game initialised')
ai = [ 'Alpha Beta regular', 'Alpha Beta TT', 'Negamax', 'Negamax Alpha Beta TT']
menu = [ '0: Alpha Beta regular', '1: Alpha Beta TT', '2: Negamax w/ Alpha-Beta Pruning', '3: Negamax Beta TT']

for item in menu:
    print(item)
    
opt = int(input('Select AI engine: '))
if not (opt >= 0 and opt <= 3):
    opt = int(input(str(opt) + ' is the wrong input, select your AI again. press 0, 1, 2 or 3'))
else:
    print('\nYou have selected the', ai[opt], 'AI')
    
opt2 = int(input('Select play option: \n0: Man Vs Machine\n1: Machine Vs Man\n2: AI vs AI\n\n'))

depth = int(input('Select search depth:\n'))
play = True
players = ['White', 'Black', 'AI']
player = 0
played = 0
counter = 0

if opt2 == 0:
    print ('Selected option 1\n', '\n1: User (w)\n2: AI (b)\n(Press Enter for Exit!)')
    player = opt2
elif opt2 == 1:
    print ('Selected option 1\n', '\n1: AI (w)\n2: User (b)\n(Press Enter for Exit!)')
else:
    print ('Selected option default\n', '\n1: AI-1 (w)\n2: AI-2 (b)\n(Press Enter for Exit!)')
    player = 2
    
board = initializeBoard()

while play:
    if counter == 0:
        printBoard(board)
    
    if fullBoard(board):
        print ('No moves left! Game ended!')
        print ('Score Black  : ' + str(EvalBoard(board) [0]))
        print ('Score White  : ' + str(EvalBoard(board) [1]))
        if EvalBoard(board) [0] > EvalBoard(board) [1]:   
            print('Player black wins')
        elif EvalBoard(board) [0] == EvalBoard(board) [1]:
            print('It\'s a tie!')  
        else:
            print('White wins')
            
        play = False
        continue

    if player == 0:
        if counter == 0:
            print ('The next move is for', players[player], 'player.\n')
        counter += 1
        
        if played == 0:
            xy = input('X,Y: ')
            menuOptions(xy) # check for chosen options

            (x, y) = xy.split(',')
            x = int(x)
            y = int(y)

            if validMove(board, x, y):
                makeMove(board, [x, y], 'w')
                print('w', (int(x), int(y), -int(x)-int(y)))

                played = 1
            else:
                print ('Invalid move! Try again!')
                continue

    if played == 1:
        xy = input('X,Y: ')
        menuOptions(xy) # check for chosen options

        (x, y) = xy.split(',')
        x2 = int(x)
        y2 = int(y)

        if validMove(board, x2, y2):
            makeMove(board, [x2, y2], 'b')
            print('b', (int(x2), int(y2), -int(x2)-int(y2)))

            state = ''
            if EvalBoard(board) [0] > EvalBoard(board) [1]:
                state = 'Win'
            elif EvalBoard(board) [0] == EvalBoard(board) [1]:
                state = 'Draw' 
            else:
                state = 'Lose'

            played = 0
            player = 1
            move_made = 1
            counter = 0
            continue
            
        else:
            print ('Invalid move! Try again2!')
            continue
            
    else: # AI 
        if counter == 0:
            print ('The next move is for', players[player], 'player.\n')
        
        start = time.time()
        moveb, movew = BestMove(board)
        if counter == 0:
            print('Thinking time: ', time.time() - start)
        counter += 1
        
        if moveb[0] != N +1:
            makeMove(board, moveb, 'w')
            makeMove(board, movew, 'b')
            print('w', (int(movew[0]), int(movew[1]), -int(movew[0])-int(movew[1])))
            print('b', (int(moveb[0]), int(moveb[1]), -int(moveb[0])-int(moveb[1])))
                
            if player == 1:
                player = 0
            counter = 0
#     
if not play:
    print('\nGame ended!')

Omega game initialised
0: Alpha Beta regular
1: Alpha Beta TT
2: Negamax w/ Alpha-Beta Pruning
3: Negamax Beta TT
Select AI engine: 3

You have selected the Negamax Alpha Beta TT AI
Select play option: 
0: Man Vs Machine
1: Machine Vs Man
2: AI vs AI

2
Select search depth:
4
Selected option default
 
1: AI-1 (w)
2: AI-2 (b)
(Press Enter for Exit!)
    _ _ _ _ _ 
   _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
   _ _ _ _ _ _ 
    _ _ _ _ _ 
The next move is for AI player.

Thinking time:  2.249765157699585
w (1, -1, 0)
b (-1, 1, 0)
    _ _ _ _ _ 
   _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
_ _ _ w _ b _ _ _ 
 _ _ _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
   _ _ _ _ _ _ 
    _ _ _ _ _ 
The next move is for AI player.

Thinking time:  1.4191932678222656
w (-2, 0, 2)
b (0, 4, -4)
    w _ _ _ _ 
   _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
_ _ _ w _ b _ _ _ 
 _ _ _ _ _ _ _ _ 
  _ _ b _ _ _ _ 
   _ _ _ _ _ _ 
    _ _ _ _ _ 
The next 